# Import Libraries

In [182]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap

# Preprocessing Functions

In [183]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

# Read Data

In [184]:
print('preparing data!!')
df2 = pd.read_csv('./data/2012_debate.csv')
df3 = pd.read_csv('./data/2013_debate.csv')
df = pd.concat([df2, df3])
# df = df.drop(['date'], axis=1)
df = df.drop(df[df.topic == 'admin'].index)
df = df.drop(df[df.transcript.str.split().map(len) < 10].index)
df['topic'] = df.apply(lambda row: preprocess(row['topic']), axis=1)

preparing data!!


In [185]:
counts = df['topic'].value_counts()
topic_counts = {topics_index_to_name_map[key]: counts[key] for key in counts.keys()}
counts_df = pd.DataFrame.from_dict(topic_counts, orient='index', columns=['count'])
counts_df

,count
"Parliament, government and politics",497
"Crime, civil law, justice and rights",473
Health services and medicine,278
International affairs,260
Economy and finance,234
"Business, industry and consumers",205
Communities and families,199
Defence,183
Energy and environment,181
Employment and training,179


In [186]:
df = df.sample(frac=1).reset_index(drop=True)
df

,topic,transcript
0,7,13. What assessment he has made of the effect...
1,8,14. How many young apprenticeship starts ther...
2,2,"I beg to move, That this House believes that ..."
3,3,18. What assessment he has made of recent tre...
4,12,(Urgent Question): To ask my right hon. Friend...
...,...,...
3583,11,10. How many requests for a reconsideration o...
3584,15,4. What steps she is taking to reduce workles...
3585,12,4. What recent representations he has receive...
3586,20,(Urgent Question): To ask the Chancellor of th...


In [187]:
X = df['transcript'].values
Y = df['topic'].values

# Preprocess Data

In [188]:
print('preprocessing data!!')
#preprocessed_X = list(map(preprocess_text, X))
preprocessed_X = parmap.map(preprocess_text, X, pm_pbar=True)
tagged_X = [models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_X)]

preprocessing data!!


3596it [00:02, 1506.92it/s]                          


# Train Doc2Vec

In [215]:
print('training doc2vec')
doc2vec_model = Doc2Vec(vector_size=100, window=3, workers=mp.cpu_count(), epochs=40)
doc2vec_model.build_vocab(tagged_X)
doc2vec_model.train(tagged_X, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
doc2vec_model.save('doc2vec')

training doc2vec


# Prepare Input Data for Neural Net Classifier

In [216]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
input_Y = Y.reshape(-1,1)
enc.fit(input_Y)
input_Y = enc.transform(input_Y).toarray()

In [217]:
print('preparing inputs')

def get_doc_vec(doc):
    return doc2vec_model.infer_vector(doc.words)

inputs = parmap.map(get_doc_vec, tagged_X, pm_pbar=True)

# for x in tagged_X:
#     topic_vec = doc2vec_model.infer_vector(x.words)
#     inputs.append(topic_vec)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, input_Y, stratify=input_Y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

preparing inputs


3596it [00:15, 230.86it/s]                          


X_train:  (2870, 100)
y_train:  (2870, 21)
X_test:  (718, 100)
y_test:  (718, 21)


# Classifier Architecture

In [218]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(21, activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

# Train Neural Network

In [219]:
model = build_network()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, batch_size=32, epochs=200, callbacks=[callback], validation_data=(X_test, y_test))

Train on 2870 samples, validate on 718 samples
Epoch 1/200
2870/2870 [==============================] - 2s 529us/sample - loss: 6.0981 - accuracy: 0.1711 - val_loss: 4.3858 - val_accuracy: 0.3148
Epoch 2/200
2870/2870 [==============================] - 0s 80us/sample - loss: 3.4250 - accuracy: 0.4453 - val_loss: 2.7048 - val_accuracy: 0.5599
Epoch 3/200
2870/2870 [==============================] - 0s 79us/sample - loss: 2.2984 - accuracy: 0.5976 - val_loss: 2.0756 - val_accuracy: 0.6295
Epoch 4/200
2870/2870 [==============================] - 0s 77us/sample - loss: 1.8704 - accuracy: 0.6645 - val_loss: 1.8352 - val_accuracy: 0.6407
Epoch 5/200
2870/2870 [==============================] - 0s 80us/sample - loss: 1.6973 - accuracy: 0.6843 - val_loss: 1.7212 - val_accuracy: 0.6657
Epoch 6/200
2870/2870 [==============================] - 0s 81us/sample - loss: 1.6060 - accuracy: 0.6941 - val_loss: 1.6637 - val_accuracy: 0.6643
Epoch 7/200
2870/2870 [==============================] - 0s 81us

# Save Model

In [220]:
model.save_weights('./topics_classifier_2')